In [204]:
import plotly.express as px
import datetime
from ast import literal_eval
from surprise import (Reader, Dataset, SVD, SVDpp, KNNBasic, NMF, CoClustering, SlopeOne,
                      NormalPredictor, KNNBaseline, KNNWithMeans, KNNWithZScore, BaselineOnly,
                      accuracy, accuracy)
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
import nltk
from nltk.stem.snowball import SnowballStemmer
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
import random

In [205]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv


In [206]:
df_tmdb_movies = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
df_tmdb_credits = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')
df_ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings.csv')
df_small_links = pd.read_csv('/kaggle/input/the-movies-dataset/links_small.csv')
df_credits = pd.read_csv('/kaggle/input/the-movies-dataset/credits.csv')
df_keywords = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')
df_movies = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv',low_memory=False)
df_ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
df_links = pd.read_csv('/kaggle/input/the-movies-dataset/links.csv')

In [207]:
df_tmdb_movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [208]:
df = df_tmdb_movies.merge(df_tmdb_credits,left_on='id',right_on='movie_id')

In [209]:
df = df.drop(['movie_id','title_y'],axis=1)

In [210]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'cast', 'crew'],
      dtype='object')

In [211]:
df = df.rename(columns={'title_x':'title'})

In [212]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew'],
      dtype='object')

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [214]:
df.shape

(4803, 22)

In [215]:
df.isna().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
dtype: int64

In [216]:
# df['homepage']
# df[df['tagline'].isna()].head()

In [217]:
df = df.drop(['homepage','tagline'],axis=1)

In [218]:
# 
df = df.drop([2656,4140],axis=0)

In [219]:
# df.shape
# (4801, 20)

In [220]:
df = df.dropna()

In [221]:
df.shape

(4799, 20)

In [222]:
top_20_movies= df.groupby('title')[['revenue','popularity','vote_average','vote_count']].max().sort_values(by =['popularity','vote_count'],ascending=[False,False]).head(20).reset_index()
top_20_movies['revenue'] = top_20_movies['revenue']/1000000
top_20_movies

,title,revenue,popularity,vote_average,vote_count
0,Minions,1156.730962,875.581305,6.4,4571
1,Interstellar,675.120017,724.247784,8.1,10867
2,Deadpool,783.112979,514.569956,7.4,10995
3,Guardians of the Galaxy,773.328629,481.098624,7.9,9742
4,Mad Max: Fury Road,378.858340,434.278564,7.2,9427
5,Jurassic World,1513.528810,418.708552,6.5,8662
6,Pirates of the Caribbean: The Curse of the Bla...,655.011224,271.972889,7.5,6985
7,Dawn of the Planet of the Apes,710.644566,243.791743,7.3,4410
8,The Hunger Games: Mockingjay - Part 1,752.100229,206.227151,6.6,5584
9,Big Hero 6,652.105443,203.734590,7.8,6135


In [223]:
fig=px.bar(top_20_movies,y='title',x='popularity',color='revenue')
fig.show()

In [224]:
fig=px.bar(top_20_movies,y='title',x='popularity',color='vote_count')
fig.show()

In [225]:
fig = px.scatter(df,x='popularity',y='revenue',color='vote_count')
fig.show()

In [226]:
df_ratings['timestamp'] = df_ratings['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x))

In [227]:
from collections import Counter

In [228]:
# df['cast'][0]

In [229]:
# df['crew'][0]

In [230]:
# import re
# def extract_genres(x):
#     patt = r'"name":\s*"([^"]+)"'
#     match = re.finditer(patt,x)
#     names = []
#     for m in match:
#         names.append(m.group(1))
#     return names

from ast import literal_eval

features = [ 'cast','crew', 'keywords', 'genres']

for feature in features:
    df[feature] = df[feature].apply(literal_eval)

# Function to get the director's name from the 'crew' column
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

# Function to get the list of top 3 elements or the entire list, whichever is more
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

# Function to clean data: Convert strings to lower case and remove spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

# Function to create a bag of words from various columns
df['director'] = df['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    df[feature] = df[feature].apply(get_list)

In [231]:
genres_count = df['genres'].apply(lambda x:','.join(x))
genres_count = Counter(', '.join(genres_count).split(', '))
df_top_20_genres = pd.DataFrame(genres_count.most_common(20),columns=['genres','total'])


fig = px.pie(df_top_20_genres,values='total',names='genres')
fig.show()

In [232]:
df['release_date']=pd.to_datetime(df['release_date'])
df['year'] = df['release_date'].dt.year

In [233]:
from sklearn.preprocessing import MinMaxScaler

In [234]:
# weighted rule formula
C= df['vote_average'].mean()
m= df['vote_count'].quantile(0.9)
df_qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notna()) & (df['vote_average'].notna())][['id','title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]

def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)
df_qualified['weighted_average'] = df_qualified.apply(weighted_rating,axis=1)

current_year=2020
df_qualified['time_decay_factor'] = 1 / (current_year - df['year'] + 1)
max_revenue = df['revenue'].max()
min_revenue = df['revenue'].min()
df_qualified['normalized_revenue'] = (df['revenue'] - min_revenue) / (max_revenue - min_revenue)
scaler = MinMaxScaler()

scaled_features = scaler.fit_transform(df_qualified[['popularity','weighted_average', 'time_decay_factor', 'normalized_revenue']])
weighted_df = pd.DataFrame(scaled_features,columns = ['popularity','weighted_average','time_decay_factor','normalized_revenue'])
weighted_df.index = df_qualified.id
weighted_df['score'] = (
    weighted_df['weighted_average'] * 0.4 +
    weighted_df['popularity'] * 0.4 +
    weighted_df['time_decay_factor'] * 0.05 +
    weighted_df['normalized_revenue'] * 0.15
)

weighted_df_sorted = weighted_df.sort_values(by='score', ascending=False)
top_10_movies = weighted_df_sorted.head(10)
result_df = pd.merge(top_10_movies, df[['id','original_title', 'year', 'revenue']], on='id', how='left')

result_df[['original_title', 'year', 'revenue', 'score']]

,original_title,year,revenue,score
0,Interstellar,2014,675120017,0.767396
1,Minions,2015,1156730962,0.662530
2,Guardians of the Galaxy,2014,773328629,0.634284
3,Deadpool,2016,783112979,0.610148
4,The Dark Knight,2008,1004558444,0.537028
5,Mad Max: Fury Road,2015,378858340,0.516111
6,Inception,2010,825532764,0.514251
7,Avatar,2009,2787965087,0.498011
8,Jurassic World,2015,1513528810,0.488658
9,The Lord of the Rings: The Return of the King,2003,1118888979,0.481762


In [235]:
# df['time_decay_factor']

In [236]:
df['overview'].isna().sum()

0

In [237]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['overview'])
tfidf_matrix.shape

(4799, 20977)

In [238]:
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)

In [239]:
indices = pd.Series(df.index,index=df['title']).drop_duplicates()

In [240]:
def get_recommendations(title,cosine_sim=cosine_sim):
    idx = indices[title]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores,key=lambda x:x[1],reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices]


In [241]:
get_recommendations('The Dark Knight Rises')

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
879                         Law Abiding Citizen
2193                       Secret in Their Eyes
1398                                  Max Payne
2416                                    Beastly
1068       The Hitchhiker's Guide to the Galaxy
979                         Free State of Jones
1202                               Legal Eagles
790                             American Sniper
2274                                   Survivor
1766                                    Case 39
Name: title, dtype: object

In [242]:
df['director'].dtypes

dtype('O')

In [243]:
df['cast']=df['cast'].apply(lambda x:[str.lower(i.replace(" ","")) for i in x])
df['director'] = df['director'].astype(str).apply(lambda x:str.lower(x.replace(" ","")))
df['director'] = df['director'].apply(lambda x: [x,x, x])

In [244]:
k = df.apply(lambda x:pd.Series(x['keywords']),axis=1).stack().reset_index(level=1,drop=True)
k.name ='keywords'

In [245]:
k = k.value_counts()
print(k[:5])
k = k[k>1]

keywords
based on novel      168
independent film    161
sex                 101
woman director       96
new york             88
Name: count, dtype: int64


In [246]:
stemmer = SnowballStemmer('english')
def filter_keywords(x):
    words = []
    for i in x:
        if i in k:
            words.append(i)
    return words

In [247]:
df['keywords'] = df['keywords'].apply(filter_keywords)
df['keywords'] = df['keywords'].apply(lambda x:[stemmer.stem(i) for i in x])
df['keywords'] = df['keywords'].apply(lambda x:[str.lower(i.replace(" ","")) for i in x])
df['soup'] = df['keywords'] + df['cast'] + df['director']+df['genres']

df['soup'] = df['soup'].apply(lambda x: ' '.join(x))

In [248]:
df['soup'][0]

'cultureclash futur spacewar samworthington zoesaldana sigourneyweaver jamescameron jamescameron jamescameron Action Adventure Fantasy'

In [249]:
df['keywords'].value_counts().head(10)

keywords
[]                                  556
[independentfilm]                    67
[womandirector]                      49
[duringcreditssting]                 20
[sport]                              19
[music]                              10
[biographi]                          10
[independentfilm, womandirector]     10
[suspens]                             7
[basedonnovel]                        6
Name: count, dtype: int64

In [250]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(df['soup'])

In [251]:
cosine_sim = cosine_similarity(count_matrix,count_matrix)

In [252]:
df = df.reset_index()
titles = df['title']
indices = pd.Series(df.index,titles)

In [253]:
get_recommendations('The Dark Knight')

3                         The Dark Knight Rises
428                              Batman Returns
3853    Batman: The Dark Knight Returns, Part 2
299                              Batman Forever
1359                                     Batman
119                               Batman Begins
1181                                        JFK
9            Batman v Superman: Dawn of Justice
2507                                  Slow Burn
210                              Batman & Robin
879                         Law Abiding Citizen
2020                                 The Rookie
205          Sherlock Holmes: A Game of Shadows
4134            Gangster's Paradise: Jerusalema
506                             Despicable Me 2
2193                       Secret in Their Eyes
4389                           The Perfect Host
1084                            The Glimmer Man
3942                                  Compadres
606                                    Blackhat
Name: title, dtype: object

In [254]:
df[df['title']=='Slow Burn'][['keywords','cast','genres']]


,keywords,cast,genres
2507,[],"[rayliotta, llcoolj, mekhiphifer]","[Mystery, Crime, Drama]"


In [255]:
df[df['title']=='Despicable Me 2'][['keywords','cast','genres']]

,keywords,cast,genres
506,"[secretag, fallinginlov]","[stevecarell, kristenwiig, benjaminbratt]","[Animation, Comedy, Family]"


In [256]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = df.iloc[movie_indices][['title','vote_count','vote_average','year']]
    vote_counts = movies[movies['vote_count'].notna()]['vote_count'].astype(int)
    vote_average = movies[movies['vote_average'].notna()]['vote_average'].astype(int)
    C = vote_average.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notna()) & (movies['vote_average'].notna())]
    qualified['vote_count'] = qualified['vote_count'].astype(int)
    qualified['vote_average'] = qualified['vote_average'].astype(int)
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False)
    return qualified

In [257]:
improved_recommendations('The Dark Knight')

/tmp/ipykernel_33/29566563.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_33/29566563.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_33/29566563.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,title,vote_count,vote_average,year,wr
96,Inception,13752,8,2010,7.774859
95,Interstellar,10867,8,2014,7.723752
1196,The Prestige,4391,8,2006,7.436744
3572,Memento,4028,8,2000,7.401913
3,The Dark Knight Rises,9106,7,2012,6.847583
119,Batman Begins,7359,7,2005,6.818644
1720,Kick-Ass,4645,7,2010,6.742772
1033,Insomnia,1148,6,2002,6.058026
629,Need for Speed,1520,6,2014,6.051606
747,Gangster Squad,1778,6,2013,6.047928


In [258]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_ratings,test_size=0.25,random_state=42)

In [259]:
df.columns

Index(['index', 'budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'title', 'vote_average', 'vote_count',
       'cast', 'crew', 'director', 'year', 'soup'],
      dtype='object')

In [260]:
reader = Reader()

In [261]:
# data = Dataset.load_from_df(df_ratings[['userId', 'movieId', 'rating']], reader)
# data.split(n_folds=5)

In [262]:
# algo = SVD()  
best_params={}
best_rmse_scores = {}
best_models = {}                                                                        
reader = Reader(rating_scale=(1, 5))
algo = SVD
# Load the data into the Surprise Dataset
train_data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)
test_data = Dataset.load_from_df(test_df[['userId', 'movieId', 'rating']], reader)
trainset = train_data.build_full_trainset()
testset = test_data.build_full_trainset().build_testset()                                         
                                                                           
# shuffle ratings if you want                                              
# random.shuffle(raw_ratings)                                                
                                                                           
# 90% trainset, 10% testset                                                                                                                                                                                                        
# Select your best algo with grid search. Verbosity is buggy, I'll fix it. 
print('GRID SEARCH...')                                                    
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005]}               
grid_search = GridSearchCV(algo, param_grid, measures=['rmse'])    
grid_search.fit(train_data)                                                 

best_models[algo.__name__]=grid_search.best_estimator['rmse']
best_params[algo.__name__]=grid_search.best_params['rmse']
best_rmse_scores[algo.__name__]=grid_search.best_score['rmse']                                  

                                                                           
# retrain on the whole train set                                           
# trainset = data.build_full_trainset()                                      
# algo.train(trainset)                                                       
                                                                           
# # now test on the trainset                                                 
# testset = data.construct_testset(trainset_raw_ratings)                     
# predictions = algo.test(testset)                                           
# print('Accuracy on the trainset:')                                         
# accuracy.rmse(predictions)                                                 
                                                                           
# # now test on the testset                                                  
# testset = data.construct_testset(test_raw_ratings)                         
# predictions = algo.test(testset)                                           
# print('Accuracy on the testset:')                                          
# accuracy.rmse(predictions)  

GRID SEARCH...


In [263]:
best_models
print(best_params)
print(best_rmse_scores)

{'SVD': {'n_epochs': 10, 'lr_all': 0.005}}
{'SVD': 0.9106969759062773}


In [264]:
new_model = SVD(n_epochs= 10, lr_all= 0.005)
new_model.fit(trainset)
predictions = new_model.test(testset)
rmse_score = accuracy.rmse(predictions)

# Calculate other accuracy measures
mae_score = accuracy.mae(predictions)
fcp_score = accuracy.fcp(predictions)

print(f"Evaluation Metrics:")
print(f"RMSE score: {rmse_score}")
print(f"MAE score: {mae_score}")
print(f"FCP score: {fcp_score}")

RMSE: 0.9039
MAE:  0.6982
FCP:  0.6353
Evaluation Metrics:
RMSE score: 0.9038728774315331
MAE score: 0.6982166306164891
FCP score: 0.6352631018179572


In [265]:
predictions_df = pd.DataFrame(predictions)
predictions_df = predictions_df.rename(columns={'uid': 'userId', 'iid': 'movieId', 'r_ui': 'actual', 'est': 'predicted'})
predictions_df['error'] = np.abs(predictions_df['actual'] - predictions_df['predicted'])

In [266]:
predictions_df.dtypes

userId         int64
movieId        int64
actual       float64
predicted    float64
details       object
error        float64
dtype: object

In [267]:
algo

surprise.prediction_algorithms.matrix_factorization.SVD

In [268]:
def hybrid_predicted_rating(userId, movieId):
    # Collaborative Filtering Prediction
    collaborative_rating = new_model.predict(userId, movieId).est
    
    # Content-Based Filtering Prediction
    sim_scores = list(enumerate(cosine_sim[movieId]))
    sim_scores = sorted(cosine_sim, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]  # Top 10 similar movies
    movie_indices = [i[0] for i in sim_scores]
    similar_movies = df.iloc[movie_indices]
    similar_movies['est'] = similar_movies['id'].apply(lambda x: new_model.predict(userId, x).est)
    content_rating = similar_movies['est'].mean()
    
    # Getting the Weighted Score
    weighted_score = weighted_df.loc[df.loc[movieId, 'id'], 'score']
    
    # Hybrid Prediction using weighted average of collaborative_rating, content_rating, and weighted_score
    final_rating = (0.5 * collaborative_rating) + (0.2 * content_rating) + (0.3 * weighted_score)
    
    return final_rating

# Predict the rating for userId=1 and movieId=500
predicted_rating = hybrid_predicted_rating(userId=1, movieId=500)
print(f"The predicted hybrid rating for userId=1 and movieId=500 is: {predicted_rating}")

The predicted hybrid rating for userId=1 and movieId=500 is: 2.052311743486812


/tmp/ipykernel_33/959008734.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

